In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/My_Research/RUSTEYE/dataset_cnn_v2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 8
EPOCHS = 80

In [3]:
train_dir = '/content/dataset_cnn_v2/train'
val_dir   = '/content/dataset_cnn_v2/valid'
test_dir = '/content/dataset_cnn_v2/test'

# Data augmentation cho train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 945 images belonging to 4 classes.
Found 202 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [4]:
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer phía trên
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [5]:
# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=15, mode='min', restore_best_weights=True)

In [7]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 61s 330ms/step - accuracy: 0.3721 - loss: 1.2528 - val_accuracy: 0.5941 - val_loss: 0.8811
Epoch 2/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 161ms/step - accuracy: 0.5310 - loss: 0.9504 - val_accuracy: 0.6485 - val_loss: 0.7996
Epoch 3/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 162ms/step - accuracy: 0.5871 - loss: 0.8925 - val_accuracy: 0.5644 - val_loss: 0.8102
Epoch 4/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 157ms/step - accuracy: 0.5891 - loss: 0.8647 - val_accuracy: 0.6238 - val_loss: 0.7571
Epoch 5/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 17s 139ms/step - accuracy: 0.5967 - loss: 0.8427 - val_accuracy: 0.6584 - val_loss: 0.8266
Epoch 6/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 16s 137ms/step - accuracy: 0.6052 - loss: 0.8136 - val_accuracy: 0.6287 - val_loss: 0.8258
Epoch 7/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 22s 182ms/step - accuracy: 0.6114 - loss: 0.8134 - val_accuracy: 0.6436 - val_loss: 0.7367
Epoch 8/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 16s 138ms/step - accuracy: 0.6306 - loss: 0

In [8]:
# Load best model
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')

In [9]:
# Đánh giá trên test set
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Lấy nhãn class
class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Accuracy theo từng class
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

26/26 ━━━━━━━━━━━━━━━━━━━━ 18s 278ms/step - accuracy: 0.5777 - loss: 0.8157

Test Loss: 0.6432
Test Accuracy: 68.78%
26/26 ━━━━━━━━━━━━━━━━━━━━ 17s 403ms/step
Accuracy: 68.78048780487805
Precision: 68.16116745333237
Recall: 68.78048780487805
F1 Score: 68.34032385663299

Classification Report:
                precision    recall  f1-score   support

 bong-troc-son     0.5435    0.4902    0.5155        51
day-cap-ri-set     0.6290    0.6842    0.6555        57
        ri-set     0.6875    0.6346    0.6600        52
       vet-nut     0.8980    0.9778    0.9362        45

      accuracy                         0.6878       205
     macro avg     0.6895    0.6967    0.6918       205
  weighted avg     0.6816    0.6878    0.6834       205


Accuracy for each class:
bong-troc-son: 0.49
day-cap-ri-set: 0.68
ri-set: 0.63
vet-nut: 0.98
